# Versuch V-So 15 – Maschinelles Lernen in der wissenschaftlichen Bildanalyse

# Einleitung

Ziel dieses Versuchs ist es die physikalische Eigenschaften
- *Position*
- *lokale Dichte*
- *Volumen*
- *Intensität*
- usw.

von Zellen mit Hilfe von tiefen neuronalen Netzwerken aus Mikroskopaufnahmen zu extrahieren. Der Versuch ist im Wesentlich in vier Phasen geteilt:
1. Vorbereitung der Trainingsdaten und Visualisierung
2. Testen von verschieden Netzwerkdesigns
3. Anwedung von einem Design auf noch unbekannte Daten
3. Berechnung der physikalischen Eigenschaften

# Hinweise

Dies ist keine Prüfungssituation!
- Zu jedem Zeitpunkt kann eine [Suchmaschiene](https://en.wikipedia.org/wiki/Web_search_engine) euer Vorankommen beschleunigen.
- Wenn ein Problem auftritt, dann sucht nach Beispiele und Lösungen auf [Github](https://github.com/), [StackOverflow](https://stackoverflow.com/) und in offizielle [Dokumentationen](https://matplotlib.org/gallery/index.html).
- Es erwartet keiner, dass ihr alle Python [Befehle](https://docs.python.org/3/library/) auswendig kennt. Es wird aber erwartet, dass ihr in kurzer Zeit euch neue Befehle heraussuchen könnt.
- Wenn ihr Fragen habt, lasst es uns wissen.
- Speichert das Notebook, wann immer ihr es für notwendig haltet.

# Vorbereitung

## Macht euch mit der Maschine vertraut:

Die für die Numerik sind folgende Komponenten von Bedeutung:
- CPU (Model)
- Arbeitsspeicher (Speicherplatz)
- Speicher (Model (von nvme0n1))
- GPU (Model)

diese Komponenten entscheiden meist darüber ob die vorgeschlagenen Rechnungen auch ausgeführt werden können und sollten im Protokoll Erwähnungen finden. Die folgenden Zellen machen es euch einfach alles notwendige herauszufinden

In [ ]:
!cat /proc/cpuinfo | \
awk -v FS=':' '                                       \
  /^physical id/ { if(nb_cpu<$2)  { nb_cpu=$2 } }     \
  /^cpu cores/   { if(nb_cores<$2){ nb_cores=$2 } }   \
  /^processor/   { if(nb_units<$2){ nb_units=$2 } }   \
  /^model name/  { model=$2 }                         \
                                                      \
  END{                                                \
   nb_cpu=(nb_cpu+1);                                 \
   nb_units=(nb_units+1);                             \
                                                      \
   print "CPU model:",model;                          \
   print nb_cpu,"CPU,",nb_cores,"physical cores per CPU, total",nb_units,"logical CPU units" \
 }' # Quelle: https://superuser.com/questions/388115/interpreting-output-of-cat-proc-cpuinfo

In [ ]:
!cat /proc/meminfo | grep MemTotal

In [ ]:
!cat /sys/class/block/nvme0n1/device/model

In [ ]:
!nvidia-smi --query-gpu=gpu_name,memory.total --format=csv

## Macht euch mit der Daten vertraut

Im Gegensatz zu vielen anderen Versuche im Rahmen des Fortgeschrittenenpraktikums, ist ein Großteil der für diesen Versuch notwendigen Daten bereits vorhanden. Ihr findet die Daten unter:

**Traininglabels (Ground Truth)** - Existierende Binärmasken.

`/DATA/GT/<Experiment Name>/`

**Traininginput** - Aufnahmen auf denen die Binärmasken basieren.

`/DATA/IN/<Experiment Name>/<Parameter>/`

**Anwendungsdatensätze** - Datensätze dir ihr analysieren sollt.

`/DATA/validation/noise1000/`

**Notebook & Benutzerdaten** - Speicherort für *.ipynb* und allen Daten die ihr im Laufe des Versuchs generiert.

`/tf/Notebooks`

#### Aufgabe:
- Lasst euch den Inhalt von den Verzeichnissen anzeigen.
- Stellt je 2 verschiedene Ebenen von 2 verschiedenen Trainingslabels und -Inputpaare in einem Diagrammen dar.

Hinweise:

- Um eine Zelle auszuführen müsst ihr `Shift+Enter` drücken.

- Mit vorangestellten `!` kann man in jupyter beliebige Programme auf dem Computer ausführen.

- Unter Linux/ MacOSX kann man sich den Inhalt von Verzeichnissen mit dem Programm `ls` und den darauf folgenden Verzeichnissnamen anzeigen lassen.

- Nutzt die unten vorgeschriebene Funktion `loadImage`.
 
- Zum Anzeigen von 2D Matrizen kann die Funktion [`imshow`](https://matplotlib.org/gallery/images_contours_and_fields/image_demo.html#sphx-glr-gallery-images-contours-and-fields-image-demo-py) des `matplotlib.pyplot`-[Modules](https://matplotlib.org/gallery/index.html) verwendet werden.

In [ ]:
%matplotlib notebook

import numpy as np # importiert das numpy Modul unter den Namen 'np'
import imread # importiert imread ohne eignenen Namen

def loadImage(filepath):
    """Liest tiff-Dateien von der Festplatte als numpy array in den Arbeitsspeicher. """
    return np.asarray(imread.imload_multi(filepath))

In [ ]:
#### Lösung

### Limitiere CUDA auf eine GPU

Im Verlauf dieses Versuchs werden wir Grafikkarten benutzen um Tensoroperationen durchzuführen. Dies soll mithilfe des [*tensorflow*](https://www.tensorflow.org/) Python Moduls passieren. Standardmäßig nutzt *tensorflow*  fast den gesamten verfügbare Grafikspeicher.

Die momentane Auslastung der Grafikkarten auf den System lassen sich mit dem Programm *nvidia-smi* überprüfen.

In [ ]:
!nvidia-smi

Über die [Umgebungsvariable](https://de.wikipedia.org/wiki/Umgebungsvariable) *CUDA_VISIBLE_DEVICES* kann man die Geräte einschränken, die für [CUDA](https://de.wikipedia.org/wiki/CUDA) Berechnungen benutzt werden dürfen.

Hinweis: 
- `%env` ist *jupyter* [*Magie*](https://ipython.readthedocs.io/en/stable/interactive/magics.html) um Umgebungsvariablen zu setzen oder anzeigen zu lassen.

In [ ]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=0

### Vordefinierte Funktionen

Um schneller eine Überblick über die eingelesen 3D-Bilder als auch über die vorhergesagten Segmentierungen zu erhalten, haben wir die Funktion `zSlicer` vorbereitet. Über den Slider unter der Funktion könnt ihr die aktuell dargestellte z-Ebene verändern. Dabei unterstützt der zSlicer eine beliebige Anzahl an 2D oder 3D Arrays als Input.

#### Aufgabe
1. Lest eine *.tif*-Datei ein und nutzt den Slider von zSlicer um durch das Volumen zu scrollen.

In [ ]:
%matplotlib notebook
import ipywidgets as widgets
import matplotlib.pyplot as plt

def zSlicer(*args):
    """Zeichnet interactive imshow Diagramme von zwei 3D oder 2D numpy Arrays."""
    num_img = len(args)
            
    num_rows = 1
    num_cols = len(args)
    
    # init plot
    fig, axes = plt.subplots(num_rows, num_cols, figsize=[9.5,5])
    
    val = 0
    aspect = 'equal'
    
    plots = []
    for i, img in enumerate(args):
        if len(args) == 1:
            ax = axes
        else:
            ax = axes[i]
        
        if img.ndim > 2:

            plots.append([ax.imshow(img[val], aspect=aspect, vmin=np.amin(img[1:]), vmax=np.amax(img[1:])), img])

        else:
            ax.imshow(img, aspect=aspect, vmin=np.amin(img), vmax=np.amax(img))
            

    def update_plot(change):
        val = change['new']
        for plot,img in plots:
            plot.set_array(img[val])
            
        return


    slider = widgets.IntSlider(value=val, max=args[-1].shape[0]-1)
    display(slider)

    slider.observe(update_plot, names='value')
        

In [ ]:
#### Lösung

## Keras Einführung

[Keras](https://keras.io/) ist eine high-level Deep Learning Bibliothek, die es euch erlaubt einfach eigene Designs für 
neuronale Netzwerke zu basteln, ohne das ihr *TensorFlow* oder gar *CUDA* Code schreiben müsst. Dabei kann Keras mit einer Vielzahl von Deep Learning Backends benutzt werden. Wir werden es heute ausschließlich mit dem Tensorflowbackend benutzen.

Am Ende dieses Versuchs wollen wir Schritt für Schritt einen [Autoencoder](http://science.sciencemag.org/content/313/5786/504/tab-pdf) erstellen, der dem Design von [UNet](http://arxiv.org/abs/1505.04597) entspricht. Dazu müssen wir uns erst einmal mit den Grundlagen vertraut machen.

### Die Keras Model API

Keras bezeichnet jedes neuronale Netzwerk unabhängig von der Architektur als [*Model*](https://keras.io/models/model/). Ein Model $M$ besteht aus ein oder mehreren Ebenen (Layern), die nichts anders als hintereinander geschaltet Funktionen darstellen. 
$$M(x) = (O \circ \ldots \circ L_2 \circ L_1 \circ I)(x) $$
Die Model API von Keras kommt mit einer Vielzahl von nützlichen Befehlen, die z.B. das Übersetzung in CUDA-Code, das Training oder aber die Anwendung des Models auf neue Probleme sehr vereinfacht.

#### Beispiel: Minimales Keras Model

Ein minimales Model hat lediglich einen Input $I$ und einen Output $O$. In dem unten gegebenen Fall wird über den Input ein *numpy* Array mit den Form ($64 \times 64 \times 1$) als Input angenommen und anschließend die Identität $\mathbb{1}:x \mapsto x$ angewandt.

In [ ]:
from tensorflow.keras.layers import Input, Lambda
from tensorflow.keras.models import Model

input_shape = (64, 64, 1)

inputs = Input(input_shape)
outputs = Lambda(lambda x: x)(inputs)

simple_model = Model(inputs=inputs, outputs=outputs)
simple_model.summary()

Um dieses Model nutzen zu können, müssen wir es für die Ausführung *kompilieren* d.h. unser Python Code wird im Hintergrund in Maschienencode umgewandelt. In Keras muss man dafür die Optimierungsfunktion (hier: [Stochastic gradient descent](https://en.wikipedia.org/wiki/Stochastic_gradient_descent)) und eine Verlustfunktion (hier: Mittlere quadratische Abweichung) angeben.

In [ ]:
from tensorflow.keras.optimizers import SGD

simple_model.compile(optimizer=SGD(), loss='mean_squared_error')

Der nächste Schritt wäre schon der Trainingsprozess. Da aber keine freien Parameter existieren (vgl. Ausgabe oben) können wir uns direkt anschauen, wie wir das Model für Vorhersagen benutzen können.

Dazu erstellen wir ein Schachbrettmuster im Format $(64 \times 64)$ und verändern anschließend die Anzahl der Dimensionen damit der Input die richtige Form für das oben definierte Modell hat. Anschließend nutzen wir das einfache Modell um eine Vorhersage zu treffen.

In [ ]:
shape = (64, 64)
test_input = np.indices(shape).sum(axis=0) % 2 # https://stackoverflow.com/a/51715491
test_input =  test_input.reshape((1,*input_shape))
print(test_input.shape)
result = simple_model.predict(test_input)
print(result.shape)

Mit der Funktion `zSlicer` können wir den Input und das Resultat der Vorhersage anzeigen lassen.

In [ ]:
zSlicer(test_input[0, ..., 0], result[0, ..., 0])

Diese Minimalbeispiel ist nicht wirklich nützlich um Binärmasken aus Mikroskopaufnahmen zu generieren. Deshalb wollen wir nun ein Netzwerk mit mehreren hintereinander geschalteten [Konvolutionen](https://towardsdatascience.com/intuitively-understanding-convolutions-for-deep-learning-1f6f42faee1) erstellen.

#### Aufgabe:
- Erstellt ein neues Netzwerk (Name ist euch überlassen) mit den oben benutzen Input Objekt und 4 drauf folgende 2D [Konvolutionen](https://keras.io/layers/convolutional/) mit dem Eigenschaften:
    - Die ersten drei Konvolutionen sollen 64 Filtern besitzen, die letzte Konvolution 32
    - Einer Kernelgröße von 3 und der [hier](https://arxiv.org/abs/1502.01852) beschriebenen Initalisierung
    - Einer "rectified linear unit" als Aktivierungsfunktionen
    - Jeder Layer soll den den gesamten Input falten und nicht nur den zulässigen Input
    
- Bastelt eine Funktion die als Parameter den `input_shape` mit den oben gegebenen Wert als Standardwert annimmt und ein komplettes (noch nicht kompiliertes) Modell als Rückgabewert.
- Lasst euch die Zusammenfassung des Netzwerks ausgeben, kompiliert das Netzwerk mit den oben angegebenen Parametern und lasst euch eine Vorhersage von dem so erstellten Netzwerk ausgeben.
- Bestimmt die Form vom Resultat.

Hinweis:
- Ihr müsst (abgesehen von der geforderten) keine einzige Funktion selbst  schreiben.
- Links sind dafür da um benutzt zu werden.

In [ ]:
#### Lösung

Fällt euch bzgl. der Form des Outputs etwas auf?

#### Aufgabe
- modifiziert die vierte Konvolution und ergänzt eine fünfte und sechste Konvolution in der oben erstellte Funktion:
    - Die vierte Konvolution soll jetzt 64 Filter haben
    - Die fünfte Konvolution soll bis auf die Filteranzahl (2) identisch mit den vorherhigen Konvolutionen sein. 
    - Die neue Ausgabekonvolution soll bis auf die Kernelgröße von 1 und der Aktivierungsfunktion eine `2DConv` mit den Standardwerten darstellen. Die Filteranzahl soll von euch so gewählt werden, dass ihr ähnlich in den Minimalbeispiel 2D-Bilder mit der gleichen Form wie die Inputbilder erhaltet. Bitte verwendet als Aktivierungsfunktion:
    $$f(x)= \frac{1}{1 +\mathrm{e}^{-x}}$$
    
    - Überprüft das Resultat eures Netzwerks mit `zSlicer` (wie im Minimalbeispiel)
    
Hinweise:
- Ihr müsst NICHT eine Aktivierungsfunktion selbst definieren.
- Benutzt Kopieren und Einfügen wann immer möglich.

In [ ]:
#### Lösung

Ihr solltet in den letzten Beispielen erkannt haben, dass die Anzahl der freien Parameter (engl. *Trainable params*) immer weiter zugenommen haben. Nun stellt sich die Frage: Wie trainieren wir eigentlich ein solches Netzwerk?

# Trainingsdaten einlesen

Wie oben bereits erwähnt, liegen die Labeldaten unter `\DATA\GT` und zugehörige Inputdaten unter `\DATA\IN`. Um diese in den Arbeitsspeicher zu laden muss man für jede einzelne Datei den Dateipfad angeben.

Weil ein manuelles Abschreiben aller Dateipfade den Zeitrahmen diese Versuchs sprengen würde, lassen wir uns alle Dateipfade ausgeben, die einem bestimmten Muster entsprechen. Dazu benutzen wir die `glob`-Funktion des gleichnamigen Moduls.

#### Beispiel

In [ ]:
import glob

filelist_labels = glob.glob('/DATA/GT/*/*')
for filepath in filelist_labels:
    print(filepath)

Das Zeichen `*` bedeutet hier eine beliebige Anzahl beliebiger Zeichen. D.h, wir erhalten mit einem Befehl alle Dateien in der zweiten Unterverzeichnissebene unter `DATA/GT/`.

#### Aufgabe

- Erstelle eine Pfadliste mit allen (.tif) Dateien in `/DATA/IN`

In [ ]:
#### Lösung

### Einfache Pfadoperationen

Operationen zur Modifikation von Dateipfaden oder Pfade insgesammt finden sich im Modul [`os.path`](https://docs.python.org/3.7/library/os.path.html) in der Python Standard Library. Wir benötigen in diesem Versuch:
- `basename` - extrahiert einen Dateiname (bzw. Namen des letzten Unterverzeichnisses, wenn ein Verzeichniss angegeben ist).
- `dirname` - gibt den Namen des Elternverzeichnis des gegebenen Pfades zurück.
- `isfile` - überprüft ob eine Datei existiert und gibt *True* oder *False* zurücl.
- `join` - fügt mehrere Zeichenketten zu einen Pfad zusammen

#### Aufgabe

- Erstelle zwei Dateilisten. Eine für alle Inputdateien und eine für die zugehörigen Labeldateien.
- Lasst euch die Anzahl aller vollständigen Input-Label-Paare ausgeben.

Hinweis: Es gibt weniger Input Dateien als Label Dateien, *Experiment Name* und *Dateiname* sind bei zueinandergehörigen Dateien identisch.

In [ ]:
#### Lösung

### Bilderausschnitte exrahieren

Ihr solltet bereits festgestellt haben, dass die Form der 3D Volumen nicht die Form von 2-dimensionalen $64 \times 64$ Bilder haben, die wir für die Vorhersage und dem Training des Netzwerks einsetzen wollen. Wir müssen aus den Bildstacks einzelne Bilder herausschneiden. Da wir die 3D Bilder als Numpy Arrays einlesen können wir Indexing benutzen.

#### Aufgabe 
- Sucht euch eine beliebiges Input/ Label Paar aus der Pfadliste aus.
- Wählt eine Positon (hier definiert durch den $z$, $y$, $x$ Index) [zufällig](http://lmgtfy.com/?q=random+integer+numpy+%20in+interval) aus dem Volumen aus.
- Schneidet jeweils ein Bild mit der Form $64\times64$ heraus und zweigt sie in einem Plot an.
- Bringe die jeweiligen Bilder auf die Form (1, 64, 64, 1) und vergleicht das Resultat des Letzten von euch erstellten Netzwerkes mit der tatsächlichen Binärmaske.

Hinweise: 
- Sowohl die Inputs als auch die Labels haben eine Übersichtsebene in z == 0.
- Falls ihr keinen Bildausschnitt erwischt mit Zellen, nutzt den zSlicer um euch das Labelbild anzeigen zu lassen und wählt manuell eine Position aus, die Zellen enthält

In [ ]:
#### Lösung

Wie erwartet ist die Vorhersage des Netzwerks sehr schlecht ...

#### Aufgabe
- Erzeugt euch zwei numpy Array mit der Form (100, 64, 64, 1) je aus einem Label und Inputbild (Batch)
- Benutzt die oben definierte Funktion `plotBatches` um euch die ersten 10 der 100 Minbatches (Untereinheit von einem Batch) anzeigen zu lassen.
- Benutzt die unten vorgegebene Funktion `cropToCells` um die Volumen nach Einlesen von der Festplatte auf einen Würfel zu reduzieren in denen sich Zellen finden lassen.

In [ ]:
def cropToCells(im_x, im_y):
    
    x_vals = np.any(im_y, axis=(0,1))
    y_vals = np.any(im_y, axis=(0,2))
    z_vals = np.any(im_y, axis=(1,2))
    x_occp = np.where(x_vals)
    y_occp = np.where(y_vals)
    z_occp = np.where(z_vals)

    # cut both volumes accordingly
    im_y_ = im_y[np.amin(z_occp):np.amax(z_occp), np.amin(y_occp):np.amax(y_occp), np.amin(x_occp):np.amax(x_occp)]
    im_x_ = im_x[np.amin(z_occp):np.amax(z_occp), np.amin(y_occp):np.amax(y_occp), np.amin(x_occp):np.amax(x_occp)]

    return im_x_, im_y_


In [ ]:
#### Lösung

### Training von einem einfachen Netzwerk

#### Aufgabe
- [Trainiert](https://keras.io/models/sequential/#fit) das oben definierte Netzwerk mit dem soeben generierten Batch. Dafür
    - Kompiliert es neu mit *Stochastic gradient descent* als Optimierungsfunktion, dem Metrikparameter `accuracy` und als Verlustfunktion die `binary_crossentropy`.
    - Startet das Training auf 5 Epochen, wobei 10 Prozent der Batches für die Validierung benutzt werden sollen.

In [ ]:
#### Lösung

Glückwunsch Ihr habt ein erstes einfaches Netzwerk erfolgreich trainiert!

#### Aufgabe
- Erstellt ein neues Batch, welches auf einem anderen Input/ Label-Paar basiert, als das Trainingsbatch und schaut euch die ersten 10 [Vorhersagen](https://keras.io/models/model/#predict) eures Netzwerks an.
- Vergleicht das Ergebnis mit der tatäschlichen Binärmaske.

In [ ]:
#### Lösung

# Bildgenerator

Wir wollen heute die von uns erstellten Netzwerke nicht nur auf einzelnen Bildern oder einzelnen Experimenten trainieren, sondern auf allen verfügbaren. Der komplette Datensatz lässt sich aber nicht in einem einzelnen *numpy*-Array zusammenfassen, der Arbeitsspeicher reicht nicht aus.

Um trotzdem die Netzwerke mit allen verfügbaren Daten zu trainieren benutzt Keras (übrigens genauso wie Python) Generatoren. Generatoren erzeugen Daten erst wenn sie aufgerufen werden.

#### Aufgabe:
 - Ergänzt den unten teilweise vorgeschriebenen Generator und Normierungsfunktionen
 - Testet den Generator mit den weiter unten stehenden Test-Code.
 
Hinweis:
- Startet mit den Normierungen, ergänzt dann die Funktionen `__init__`, `__getimages__` und zum Schluss `__getitem__`.

#### Normalizations

In [ ]:
import numpy as np

def zeroMeanUnitVariance(a, axis=None):
    """
    # Aufgabe: Schreibt eine eigene Norm-Funktion die das Inputarray 'a' auf gegebenen Achsen auf einen
    Durchschnittwert von 0 und einer Varianz von 1 normiert.
    
    Hinweis: Benutzt numpy Funktionen die als Schlüsselwörter die Achse akzeptieren und 
    für leichtere Implementierung die Dimensionen des Inputs erhalten.
    
    """
    
    return a

def zeroToOne(a, axis=None):
    """
    # Aufgabe: Schreibt eine  Norm-Funktion die das Inputarray 'a' auf gegebenen Achsen auf das Intervall
    [0, 1] normiert.
    
    Hinweis: Benutzt numpy Funktionen!
    
    """
    a = a - np.amin(a, axis=axis, keepdims=True)
    return a / np.amax(a, axis=axis, keepdims=True)
    

In [ ]:
from tensorflow.keras.utils import Sequence
from tensorflow.keras.utils import to_categorical
from tensorflow.image import grayscale_to_rgb
import glob
import os
import numpy as np
import imread

def loadImage(filename):
    return np.asarray(imread.imload_multi(filename))

# Code (heavily) modified from https://stanford.edu/~shervine/blog/keras-how-to-generate-data-on-the-fly
class DataGenerator(Sequence):
    
    def __init__(self, gt_path = '/DATA/GT', in_path = '/DATA/IN', setname='set*', snr=1000, snrname='dz400_IMax{}',
                 patches_per_img=32, img_per_epoch=1000, batch_size=32, dim=(64,64), n_channels=1, n_classes=1,
                 use_fraction=None, norm=zeroMeanUnitVariance, shuffle=True):
        
        
        #self.snr = snr
        self.setname = setname
        
        self.patches_per_img = patches_per_img
        self.img_per_epoch  = img_per_epoch
        self.batch_size = batch_size
        self.dim = dim
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.norm = norm
        
        
        """
        # Aufgabe: Ergänzt den Programmcode zum Erstellen der Dateilisten
        
        Zu benutzende Variablen:
        gt_path - Elternverzeichnis von Experimentordnern (vgl. Standardwert)
        in_path - Großelternverzeichnis von Experimentordnern (vgl. Standardwert)
        setname - Name des Experimentordners (vgl. Standardwert, soll * benutzen können)
        snr - Signal zu Rauschverhältnis (vgl. STandardwert, Teil des Parameterordnernames von Input)
        snrname - Formatstring in dem snr eingesetzt werden soll (vgl. "Using % and .format() for great good!")
        
        
        Hinweis: Die Standardwerte sind bereits als Parameter von __init__ gegeben.
        
        
        Zu erstellende Variablen:
        
        x_filelist - Liste von Dateipfaden (Strings) zu existierenden Inputbildern
        y_filelist - Liste von Dateipfaden (Strings) zu existierenden Labelbildern
        
        """
        
        
        # Store filelist in member field
        
        self.x_filelist = x_filelist
        self.y_filelist = y_filelist
        
        print(len(y_filelist))
        
        self.list_IDs = np.arange(self.y_filelist.shape[0]*self.patches_per_img)
        self.indexes = np.arange(len(self.list_IDs))
        
    def __len__(self):
        'Number of batches per epoch'
        return int(np.floor(self.y_filelist.shape[0]*self.patches_per_img / self.batch_size)) 

    def __getitem__(self, index):
        'Generate one batch of data'
        # Select image from image list (Tweak: multiple images)
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size] # shuffeled id list

        list_IDs_temp = [self.list_IDs[k] for k in indexes] # ids in planned number of images per epoch

        X = np.empty((self.batch_size, *self.dim, self.n_channels))
        Y = np.empty((self.batch_size, *self.dim, self.n_classes), dtype=bool)
        
        file_ids = np.arange(self.y_filelist.shape[0])
        file_ids = np.repeat(file_ids, self.patches_per_img)
        
        previous_file_id = -1
        
        for i, file_id in enumerate(file_ids[list_IDs_temp]):
            
            if previous_file_id != file_id:
                im_x, im_y, shape =  self.__getimages__(file_id)
                    
            # else: work with existing im_x, im_y
            
            """
            # Aufgabe: Ergänzt eine Funktion, die aus im_x und im_y die Minibatches für Input und Label in 
            der unten beschriebenen Form an zufällig Positionen ausschneidet und für das Batch vorbereitet.
            
            
            Zu benutzende Variablen
            self.norm - Normierungsfunktion von Input-Minibatch (wie oben definiert)
            self.dim - y, x Ausdehnung der Minibatches (wie oben definiert)
            im_x - Eingesenes Input Volumen mit den Dimensionen (z, y, x)
            im_y - Eingelesenes Label Volumen mit den Dimensionen (z, y, x)
            shape - Form der beiden eingelesenen Volumen
            
            
            Zu erstellende Variablen:           
            im_x_norm - auf die Form (self.dim[0], self.dim[1], 1) zurechtgeschnittenes und normiertes Minibatch
                        von im_x.
            im_y_norm - auf die Form (self.dim[0], self.dim[1], 1) zurechtgeschnittenes Minibatch von im_y.
            """
            
            # stacks input for RGB networks
            im_x_norms = ()
            for _ in range(self.n_channels):
                im_x_norms = im_x_norms + (im_x_norm,)
            im_x_norm = np.concatenate(im_x_norms, axis=-1)
                
            X[i, ...] = im_x_norm
            Y[i, ...] = im_y_norm
            
            previous_file_id = file_id
            

        return X, Y
    
    def __getimages__(self, file_id):
        """ Reads .tif Files and cuts them to a cube where Cells can be found."""
        
        """
        # Aufgabe: Ergänzt das Einlesen von Dateien
        
        Zu verwendende Funktionen (Bereits in Notebook definiert)
        loadImage() 
        cropToCells()
        
        
        
        Benötigte Variablen:
        
        im_x - eingelesene Input .tif Datei als numpy Array
        im_y - eingelesene Label .tif Datei als numpy Array
        im_x_ - auf Würfel mit Zellen zurechtgeschnittenes Input Array  
        im_y_ - auf Würfel mit Zellen zurechtgeschnittenes Label Array
        shape - Zellwürfel Form
        
        """

        if not np.any(np.asarray(shape) - np.array([0, self.dim[0], self.dim[1]]) <= 0): # cutted array large enough
            im_y = im_y_
            im_x = im_x_
        else:
            shape = im_y.shape

        return im_x, im_y, shape

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        if self.shuffle == True:
            # random offset
            offset = np.random.randint(0, len(self.list_IDs) )
            indexes = np.empty_like(self.list_IDs)
            indexes[:-offset] = self.list_IDs[offset:]
            indexes[-offset:] = self.list_IDs[:offset]
            
            # random order of chunks with length of batch size
            chunk_order = np.arange(self.__len__())
            np.random.shuffle(chunk_order)
            self.indexes = [i for index in chunk_order for i in indexes[index*self.batch_size:(index+1)*self.batch_size]]
        else:
            pass
    

In [ ]:
#### Lösung

In [ ]:
#### Testcode 1
# Erstelle eine neue DataGenerator Instanz
beispiel_generator = DataGenerator()

In [ ]:
#### Testcode 2
# Erstellt ein Beispielbild
images = beispiel_generator.__getimages__(1)[:2]
# Zeigt die Beispielbilder an
zSlicer(*images)

In [ ]:
#### Testcode 3
# Erstellt ein Beispieltrainingsset
batches = beispiel_generator.__getitem__(0)
# Zeigt das Beispieltrainingsset an
plotBatches(*batches)

## Netzwerktraining mit Generator Tensorboard-Callback

Wir haben jetzt die Möglichkeit alle Mikroskopaufnahmen zum Training zu benutzen. 

#### Aufgabe:
- Erstelle ein neues Netzwerk mit dem oben benutzten Design.
- Kompiliert das Netzwerk mit *stochastic gradient descent* als Optimierungsfunktion,  `binary_crossentropy` als Verlust Funktion und der `accuracy`-Metrik.
- [Trainiert](https://keras.io/models/model/#fit_generator) das Netzwerk mit
    - Einer neuen `DataGenerator`-Instanz 
        - mit `setname='set9*'`
        - und `use_fraction=0.5`
    - Auf 20 Epochen
    - Nutzt dabei Multiprocessing und 6 worker um das Erstellen der Minibatches zu beschleunigen.
    - Erstellt eine Variable `history` die den Trainingsprozess als Wert zugewiesen bekommt.

Hinweis: Der Traininsprozesses dauert auf der verbauten Grafikkarte ca. 10 min.

In [ ]:
#### Lösung

In der Variable `history` haben wir Informationen über den Trainingsprozess gespeichert.
Mit diesem Informationen wollen wir ein Diagramm erstellen, das den Verlauf des Trainingsprozesses über mehrere Epochen darstellt.

#### Beispiel

In [ ]:
# Data preparation
Y = [history.history['acc'], history.history['loss']]
YLabels = ['Genauigkeit', 'Verlust']

# Plotting
f, axes = plt.subplots(1,2, figsize=(9.5, 5))

for ax, y, ylabel in zip(axes, Y, YLabels):
    ax.plot(np.arange(1, len(y)+1), y, 'ro', label='6x Conv2D')
    ax.set_xlabel('Epoche')
    ax.set_ylabel(ylabel)
    ax.legend()
    ax.grid()

plt.tight_layout()

### **Bitte sammelt alle Trainigsverläufe in entsprechenden Dateien!**

Um euch die Arbeit zu erleichtern, haben eine Funktion vorgeschrieben, welches nicht nur den Trainingsverlauf, sondern auch das fertig trainierte Modell abspeichert. Sicher ist sicher ...

In [ ]:
from datetime import datetime
import csv
import os

def save_model_history(history, net, foldername):
    
    folder = os.path.join(foldername, datetime.now().strftime('%Y-%m-%d_%H-%M'))
    
    if not os.path.exists(folder):
        os.makedirs(folder)
        
    hist_pivot = [dict(zip(history.history, col)) for col in zip(*history.history.values())] # https://stackoverflow.com/a/37489316
        
    with open(os.path.join(folder, 'history.csv'), 'w') as f:
        w = csv.DictWriter(f, history.history.keys())
        w.writeheader()
        for row in hist_pivot:
            w.writerow(row)
        
    
    
    model_json = net.to_json()
    with open(os.path.join(folder, "model.json"), "w") as json_file:
        json_file.write(model_json)
        
    net.save_weights(os.path.join(folder, "weights.h5"))

#### Beispiel

In [ ]:
save_model_history(history, net, 'simple2conv')

Tatsächlich gibt es eine noch elegantere Variante den Trainingsprozess im Auge zu behalten: *Tensorboard*.
Mit *Tensorboard* kann man alle Trainingsprozesse (auch mehrere gleichzeitig) beobachten.

Der Keras Callback `Tensorboard` ermöglicht erstellt dabei die notwendigen log-Dateien. Der Callback lässt sich sehr einfach in das Training integrieren. Dazu müsst ihr denn Callback zunächst mit einem (am besten eindeutigen) Verzeichnisnamen unterhalb von `./logs` erstellen:

``` python
from datetime import datetime
from tensorflow.keras.callbacks import TensorBoard

datestr = datetime.now().strftime('%Y-%m-%d_%H-%M')
tboard = TensorBoard(log_dir='./logs/{}_6x_Conv2D'.format(datestr), update_freq='batch')
```

In der Funktion `fit_generator` müsst ihr nur noch das Schlüsselwort `callbacks=[tboard]` ergänzen. Der Callback zeichnet automatisch den Trainingsprozess auf. Der im Hintergrund bereits von uns gestartete Tensorboardprozess sammelt die Informationen und zeigt Sie auf einer eigenen [Website](http://localhost:6006) an.

#### Aufgabe
- Startet das letzte Training mit einem Tensorboard Callback erneut und beobachtet den Graphen auf der Tensorboard Website

In [ ]:
#### Lösung

# Schritt für Schritt zum Autoencoder

Die Genauigkeit aller bisher trainierte Netzwerke ist nicht zufriedenstellend.

Unser Modell soll bis zu 98% Genauigkeit erreichen. Die bisher verwendete Architektur ist dafür nicht geeignet.

Damit dies möglich wird, muss das Netzwerk *tiefer* werden d.h. wir benötigen mehr Ebenen zwischen Input und Output. Gleichzeitig wollen wir Informationen von benachbarten Pixeln zusammenfassen. Dazu werden Pooling Ebenen wie `MaxPooling2D` benutzt. Im Gegensatz zu Konvolutionen modifiziert Pooling nicht die Filteranzahl. Das Gegenstück zu `MaxPooling2D` stellen Upsampling Ebenen wie `Upsampling2D` dar.

#### Aufgabe:
- Erstellt basierend auf der bereits erstellten Funktion eine neue Funktion die euch fertige Modelle zurückgibt:
    1. `Input` (Parameter wie oben)
    2. 2x `Conv2D` (Parameter wie oben) \[1\]
    3. `MaxPooling2D` (mit `pool_size=(2,2)`)
    4. 2x `Conv2D` (Parameter wir oben nur doppelter Filter Anzahl)
    5. `Upsampling2D`(`size` wie in `MaxPooling2D`)
    6. `Conv2D` (Parameter wie oben nur Kernelgröße von 2) \[2\]
    7. `Concatenate`(entlang der Filter Dimension mit einer Liste mit \[1\] und \[2\] als Input)
    8. 2x `Conv2D` (Parameter wie oben)
    9. 2x `Conv2D` (wie die letzten beiden Konvolutions oben)
- Testet das Model auf 10 Epochen mit
    - Trainingsdatengenerator mit `setname='set1*'`
    - Validierungsgenerator mit `setname='set9*'` und `use_fraction=0.20`

In [ ]:
#### Lösung

Das neue Design bleibt hinter den Erwartungen zurück. Durch noch tiefere Netzwerke können wir die Genauigkeit auf den angestrebten Wert steigern.
#### Aufgabe
- Schreibt eine Funktion für den MaxPooling & Convolution Schritt. (3. und 4.)
- Schreibt eine Funktion für den UpSampling und Convolution Schritt (5., 6., 7. und 8.)
- Erstellt Netzwerke mit 2, 3, 4 MaxPooling Schritte (und der selben Anzahl an UpSampling Layer)
- Fügt zusätzlich in die Netzwerke mit 3, 4 Pooling Ebenen eine Dropout Ebene nach dem 3. und 4. Pooling Schritt ein.
- Die 'dropout_rate' von den Dropout Ebenen soll 0.5 betragen.

In [ ]:
#### Lösung

Je mehr Ebenen unsere Modelle beinhalten, umso schwieriger wird es die Anpassung der Fehlerfunktion auf Ebenen am Anfang des Netzwerks weiterzugeben. Je besser dies gelingt, desto schneller konvergieren die Netzwerke in einen Gleichgewichtszustand, der möglichst ähnliche Resultate für ähnlichen Input garantiert. Um die Konvergenz der Netzwerke zu erleichtern, kann man eine andere Optimierungsfunktion wie z.B. [Adam](https://arxiv.org/pdf/1412.6980v8.pdf) benutzen.

Der [Dropout](https://arxiv.org/pdf/1207.0580.pdf) Ansatz reduziert die für das Training benutzte Neuronen adaptiv um das Modell robuster zu machen.

#### Aufgabe:
- Trainiert alle oben angegeben Netzwerke (inkl. des Netzwerks welches nur 6 Konvolutionen benuzt) mit der `Adam` Optimierungsfunktion anstatt `SGD`.
- Benutzt dabei eine *Learning Rate* von 1e-4.
- 20 Epochen
- Vergleicht die Performance der Netzwerke.

Hinweis:
- Schreibt eine Schachtelung von For-loops die die Aufgaben unmittelbar nacheinander ausführt.
- Erstellt euch mit Hilfe der Funktion `create_batches_ram` einmalig ein Trainings - und Validationset und verzichtet bei diesem Benchmark auf die direkte Nutzung des Generators während des Trainingsprozesses. 

In [ ]:
def create_batches_ram(generator):
    tmp_batch = generator.__getitem__(0)[0]
    batch_size = tmp_batch.shape[0]
    num_batches = generator.__len__()
    
    X = np.zeros((num_batches*batch_size, 64, 64, 1))
    Y = np.zeros((num_batches*batch_size, 64, 64, 1))
    
    
    for i in range(num_batches):
        out = generator.__getitem__(i)
        X[i*batch_size:(i+1)*batch_size] = out[0]
        Y[i*batch_size:(i+1)*batch_size] = out[1]
    
    return X, Y

In [ ]:
#### Lösung

# Treffe Voraussagen auf noch unbekannte Daten

Im nächsten Teil des Praktikums wollen wir uns mehr auf die Eigenschaften der Zellen innerhalb der Volumen konzentrieren. 

- Welche physikalischen Zelleigenschaften können wir aus noch unbekannten Mikroskopaufnahmen extrahieren.

Dazu benutzen wir die Mikroskopaufnahmen unter `/DATA/validation/noise1000/`. Im Folgenden wollen wir nur das Netzwerk mit der höchsten Genauigkeit auf den Validierungsdaten verwenden.

In [ ]:
!ls /DATA/validation/noise1000/

Sowohl der Trainingsgenerator, als auch der Validationgenerator erstellt an zufälligen Positionen im Volumen 2D Minibatches mit der Größe von $64 \times 64$ Pixel.

Diese Volumenausschnitte haben auf den ersten Blick wenig mit einer Vorhersage eines kompletten Volumens zu tun. Dazu muss das Volumen in kleine Teilbilder zerschnitten werden. Dabei ist es wichtig, dass eine festgelegte Reihenfolge eingehalten wird, damit hinterher die Vorhersagen wieder zu einem Volumen zusammengesetzt werden können.

### Aufgabe:
- Schaut euch die unten definierte Funktion `predictImageStack`an.
- Benutzt die Funktion um Zell vorhersagen der oben angegeben Mikroskopbilder anzufertigen.
- Berechnet von jeder Zellvorhersage den Durchschnitt in $x$, $y$ und $z$-Richtung. Fällt euch etwas auf?

In [ ]:
def predictImageStack(img, model, num_channels=1, num_classes=1, batch_size=32, norm=zeroMeanUnitVariance):
    print('Image shape: {}'.format(img.shape))
    input_shape = model.layers[0].output_shape
    print('Model input shape: {}'.format(input_shape))
    
    in_z, in_y, in_x = img.shape
    height, width = input_shape[1:3]
    nz = in_z - 1
    ny = int(np.floor(in_y / height))
    nx = int(np.floor(in_x / width))
    
    print('Number of tiles in x, y, z: ({:d}, {:d}, {:d})'.format(nx, ny, nz))
    
    margin_x = (in_x - (width * nx)) / 2
    margin_y = (in_y - (height * ny)) / 2
    
    
    img = img[..., None]
    imgs = ()
    for _ in range(num_channels):
        imgs = imgs + (img,)
    img = np.concatenate(imgs, axis=-1)
    
    # cut away overview plane in z and reduce size in x, y to fit model input
    img_ = img[1:, int(np.floor(margin_y)):-int(np.ceil(margin_y)), int(np.floor(margin_x)):-int(np.ceil(margin_x)), :]
    
    print('cut image to shape: {}'.format(img_.shape))
    
    num_batches = int(nx * ny * nz)
    
    print('Total number of tiles: {}'.format(num_batches))
    
    prediction_batches = np.zeros((num_batches, height, width, num_channels))
    
    print('Prediction batches shape: {}'.format(prediction_batches.shape))
    
    
    print('Fill input batches with image data')
    k = 0
    for i in range(ny):
        for j in range(nx):
            for l in range(nz):
                prediction_batches[k, :, :, :] = norm(img_[l, height*i:height*(i+1), width*j:width*(j+1)], axis=(0, 1, 2))
                k += 1
                
    num_predictions = num_batches // batch_size
    
    print('Predict batches')
    for i in range(num_predictions):
        prediction_batches[i*batch_size:(i+1)*batch_size] = model.predict(prediction_batches[i*batch_size:(i+1)*batch_size])
    
    if num_predictions*batch_size != num_batches:
        prediction_batches[num_predictions*batch_size:] = model.predict(prediction_batches[num_predictions*batch_size:])
    
    img_prediction = np.zeros((*img_.shape[:3], num_classes))
    
    print('Constuct predicted image with shape: {}'.format(img_prediction.shape))
    
    k = 0
    for i in range(ny):
        for j in range(nx):
            for l in range(nz):
                img_prediction[l, height*i:height*(i+1), width*j:width*(j+1)] = prediction_batches[k, ..., :num_classes]
                k += 1

    
    return np.squeeze(img_prediction), np.squeeze(img_)


In [ ]:
#### Lösung

## Isotropische Auflösung

Um wirklich quantitative Aussagen über Zellen zu treffen (Volumen, Intensität, Abstände), müssen wir beachten, dass
in unterschiedliche Raumrichtungen unterschiedliche Auflösungen existieren. In den Beispielbildern wird von einem Pixelabstand von $63,2\,\mathrm{nm}$ in $x$ und $y$-Richtung und von $400\,\mathrm{nm}$ in $z$-Richtung verwendet.

Für isotropische Auflösungen wollen müssen wir den Pixelabstand in $z$ Richtung auf ebenfalls $63,2\,\mathrm{nm}$ linear interpolieren.

Dazu bietet sich die Funktion `affine_transform` im Modul `scipy.ndimage` an. Die benötigten Parameter sind:
 - das Usprungsvolumen (ohne Übersichtsebene in `[0, :, :]`)
 - die Transformationsmatrix (eine $3 \times 3$ Diagonalmatrix, die auf der Hauptdiagonale die Einträg $\left(d, 1, 1\right)$ besitzt, wobei $d$ der gewünschte Streckungsfaktor in $z$-Richtung darstellt).
 - die neue Form des Volumens mit bereits angepasster Anzahl an $z$-Ebenen
 - `order=1` welches die Interpolation auf die lineare Ordnung beschränkt.

#### Aufgabe
- Berechnet die lineare Interpolation von allen .tif Dateien in `/DATA/validation/`
- Speichert die Interpolationen als `.npy`-Datei auf der Festplatte. 


In [ ]:
#### Lösung

#### Aufgabe: 
- Schaut euch die Dateigröße der eben gespeicherten Dateien an.
    - Dazu können ihr beispielsweise Programm `du` benutzen.
    - Input ist der Verzeichnispfad.
    - Die Optionen `-hs` erstellt eine gut lesebare Zusammenfassung. 

In [ ]:
#### Lösung

Die Dateien sind bis zu mehrere Gigabyte groß.

Bei jeder folgenden Rechenoperation muss die gesamte Datei in den Arbeitsspeicher geladen werden. Deshalb wollen wir uns schnell einen Überblick verschaffen, ob wir tatsächlich alle Voxel (=Pixel in 3D) benötigen.

#### Aufgabe
- Erstellt eine Maximumsprojektion aller 3 Raumachsen
- Lasst euch die Projektionen anzeigen und benutzt nur den Teil des Volumens, der Zellen beinhaltet.
- Speichert die so reduzierten Volumen als `.npy`-Dateien ebenfalls auf der Festplatte und vergleicht die Dateigrößen.

In [ ]:
#### Lösung

Die Dateigröße konnte sehr stark reduziert werden. Dies ist gängig Praxis bei Mikroskopdaten, bei denen nur ein bestimmter Bildausschnitt (Region of Interest) für die Analyse interessant ist. Das Verkleinern des Bildausschnitts auf die ROI nennt man *Cropping*. Dies hat vor allem ressourcetechnische Vorteile:
1. Die Dateien brauchen weniger Speicherplatz.
2. Die Analyse auf den Daten geht schneller.

Zu Bedenken ist aber auch, dass dies die Möglichkeit eröffnet nur Bildausschnitte zu veröffentlichen, die die eigenen Thesen untermauern. Deshalb gilt es als gute wissenschaftliche Praxis die *Primärdaten* ebenfalls aufzuheben.

**Bitte speichert alle Ergebnisse, für die Ihr die Grafikkarten zur Vorhersage benutzt habt auf der Festplatte. Achtet dabei bitte auf eine eindeutige Benennung bzw. schreibt euch alle Schritte in eurem Laborbuch auf. Denkt daran, dass ihr auf den Daten auch im Nachhinein noch Analysen für das Protokoll anfertigen müsst.**

#### Aufgabe
- Nutzt ein Netzwerk um die Zellen in den Volumen zu erkennen.
- Speichert als `.npy`-Datei:
    - die Zellvorhersagen
    - die zuerechtgeschnitten Intensitätsbilder
- Benutzt die Funktion `zSlicer` um einen Input mit der Vorhersage zu vergleichen
- Erstellt ein Histogramm der Pixelwerte in der Vorhersage

Hinweis: Benutzt für das Histogram logarithmische Auftragung auf der y-Achse und mindestens 100 Abschnitte.

In [ ]:
#### Lösung

# Auswertung

**Die Auswertung ist mit den gespeicherten Werten auch Zuhause oder im Lernzentrum möglich.**


Mit der Vorhersage wollen wir jetzt weiterarbeiten. Momentan stehen wir noch vor einigen Problemem:
1. Die Vorhersage muss diskretisiert werden (von \[0, 1\] zu \{0, 1\})
2. Wir wollen Objekte, die nicht miteinander verbunden sind mit unterschiedlichen [Label](http://scikit-image.org/docs/dev/auto_examples/segmentation/plot_label.html#sphx-glr-auto-examples-segmentation-plot-label-py) versehen 

#### Aufgabe:
- Berechnet ein Volumen in dem jedes Objekt ein eigenes Label bekommt.

Hinweise: 
- In dem gegebenen Beispiel heißt das mit Label versehene Volumen `label_image`.
- Nicht alle Operationen werden in unserem Fall benötigt
- Benutzt nach Befarf `binary_erosion`, `ball`, `binary_dilation` um kleine Objekte aus eurem Datensatz zu filtern

In [ ]:
#### Lösung

Diesen segmentierten Daten kann man schon eine Reihe von Parameter extrahieren. Im Detail sind dies:
- Zellpositionen (Beispiel)
- Mittlere/ Maximale/ Minimale  Helligkeit 
- Zellvolumen
- Lokale Dichte
- Zellabstände (optional)
- Nemantic Orderparamter (optional)

## Zellposition

#### Beispiel

Sei `w` die oben bestimmte Labelmatrix (bzw. das Labelvolumen) und `img_` die der Segmentierung zugehörige Intensitätswerte, dann lässt sich ein numpy Array `centroids_array` mit allen Schwerpunkten aller Objekten in der Labelmatrix bestimmen via:

In [ ]:
from skimage.measure import regionprops

props = regionprops(w, img_)
centroids_list = [props[i].centroid for i in range(len(props))]
centroids_array = np.asarray(centroids_list)

Dabei beinhaltet das Ergebnis von `skimage.measure.regionprops` noch eine ganze Reihe von weiteren [Objekteigenschaften](http://scikit-image.org/docs/dev/api/skimage.measure.html#regionprops).

## Helligkeitswerte

#### Aufgabe
- Bestimmt die mittlere, maximale und minimale Helligkeit aller Objekte in dem gelabelten Volumen.
- Erstellt je ein Histogram für jeden der drei Helligkeitsverteilungen.

In [ ]:
#### Lösung

## Zellvolumen

Auch das Volumen lässt sich auf diese Art und Weise bestimmen, wobei der entsprechende Parametername von `props` nicht direkt ersichtlich ist.

#### Aufgabe
- Finde den entsprechenden Parametername heraus und stelle die Verteilungen der einzelnen Volumen in metrischen-Einheiten in einem Diagramm dar

In [ ]:
#### Lösung

## Lokale Dichte

Obwohl Position, Helligkeit und Volumen interessante Zellparameter sind, können wir aus den Segmentierung noch wesentlich mehr Informationen extrahieren. In den meisten Veröffentlichungen konzentiert man sich auf einige wenige Eigenschaften. Weil keine andere Arbeitsgruppe diese Eigenschaften bereits untersucht hat, muss man sich selbst überlegen wie man die Messwerte aus den Bildern extrahieren kann.

Ein solches Beispiel wäre die lokale Dichte, die man nicht in den `regionprops` Eigenschaften findet. Stattdessen sind eigene Programmierkenntnisse notwendig.

In [ ]:
from numpy.linalg import norm

def calculateLocalDensity(w, centroid_array, radius = 60):
    """ Calculates occupied volume in a sphere around centroid
    """

    Z, Y, X = np.meshgrid(np.arange(-radius, radius, 1), np.arange(-radius, radius, 1), np.arange(-radius, radius, 1))
    Z, Y, X = np.ravel(Z), np.ravel(Y), np.ravel(X)
    sphere = norm([Z, Y, X], axis=0) < radius

    w_ = w > 0
    w_ = np.pad(w_, radius, 'constant', constant_values=-1)

    vol_sphere = np.sum(sphere) 

    localDensity = np.zeros(len(props))
    for i in range(len(props)):
        z, y, x = centroid_array[i]
        z, y, x = np.round([z, y, x])
        w_part = w_[int(z):int(z+2*radius), int(y):int(y+2*radius), int(x):int(x+2*radius)]
        vol = w_part.flatten()*sphere
        out = np.sum(w_part == -1)
        localDensity[i] = np.sum(vol==1)/(vol_sphere - out);

    return localDensity

# Visualisierung


Wirklich interessant werden die einzelnen Parameter erst, wenn man die räumliche Verteilung kennt. Dazu eigenen u.A. 3D Scatterplots.

In der *matplotlib* Dokumentation findet ihr ein sehr einfaches [Beispiel](https://matplotlib.org/examples/mplot3d/scatter3d_demo.html) dazu.

#### Aufgabe
- Erstellt ein 3D Scatterplot für die durchschnittliche Helligkeit und die lokale Dichte.
    - Die Markerposition soll durch den jeweiligen Schwerpunkt gegeben sein.
    - Die Markerfarbe soll den Parameter darstellen.
    
Hinweis: Berechnet die lokale Dichte mit der gegebenen Funktion `calculateLocalDensity`. Passt dabei den Radius auf ein sinnvolles Maß an. (Dabei kann ein Blick in die Volumenverteilung nicht schaden)

In [ ]:
#### Lösung

# 3D Rendering

Zum Abschluss stehen euch zwei Möglichkeiten offen 3D Visualisierungen der Zellen zu erstellen.

## *matplotlib*

Kann in diesem Notebook gemacht werden, dauert aber lange und die Plots lassen sich nicht wirklich interativ drehen.

**Bitte entfernt nur die Anführungszeichen, wenn ihr 5min warten könnt!**

In [ ]:
from skimage.measure import marching_cubes_lewiner

verts, faces, normals, values = marching_cubes_lewiner(dilate, 0.5)

"""
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot_trisurf(verts[:, 0], verts[:,1], faces, verts[:, 2],
                cmap='Spectral', lw=1)

plt.show()
"""

## *ParaView*

ParaView  ist ein 3D Rendering Programm für große Datensätze und de facto Standard für die Visualisierung von aufwendigen Simulation. Die folgende Handreichung soll euch dazu ermuntern ParaView zum Rendern einer 3D Darstellung zu benutzen. (beispielsweise kann man mit ParaView Abbildungen wie z.B. in der [Versuchsbeschreibung](https://www.uni-marburg.de/de/fb13/studium/praktika/praktika-fuer-physiker/v-so-15-maschinelles-lernen) erstellen oder gar ganze Simulationsreihen in einen [Film](https://static-content.springer.com/esm/art%3A10.1038%2Fs41567-018-0356-9/MediaObjects/41567_2018_356_MOESM3_ESM.mp4) umwandeln).

In [ ]:
# Export to VTI
from pyevtk.hl import imageToVTK

imageToVTK('{}_unet_prediction'.format(datestr),
           pointData={"prediction":prediction})

## Download

Ladet bitte den ParaView Installer von der offiziellen [Website](https://www.paraview.org/download/) herunter. Im folgenden wird die Windowsversion benutzt. Für Max OSX und der Linux Distribution eurer Wahl finden sich entsprechende Versionen im Mac App Store und der Paketverwaltung.

## Datentransfer

Um die Daten auf euren lokalen Computer herunterladen zu können müsst ihr ein Programm benutzen, dass Daten über eine [SSH](https://de.wikipedia.org/wiki/Secure_Shell)-Verbindung übertragen kann. Unter Mac OSX und Linux könnt ihr euren normalen Dateimanager benutzen. Unter Windows benötigt ihr ein zusätzliches Programm (z.B. [WinSCP](https://winscp.net/eng/download.php)).

Für eine Verbindung mit WinSCP benötigt Ihr folgende Daten:
* File protocol: SFTP
* Host name: ***
* User name: ***
* Passwort: ***

Unter nautilus (Linux Dateimanager) findet ihr das entsprechende Menü unter 'Other Locations' und 'Connect to Server'. In die Addresszeile müsst ihr eintragen:
`sftp://<username>@<hostname>/`

Bitte ladet die `.vti`-Dateien unter `/home/<username>/Notebooks` herunter

## Datenvisualisierung in ParaView

* Öffnet die Datei `JJJJ-MM-DD_hh-mm_unet_prediction.vti`.
* Nutzt den Threshold Filter im Bereich eurer Labels.
* Spielt ein bisschen mit den Einstellungen im Bereich *OSPRay Rendering* bis ihr eine schöne 3D Visualizierung erstellt habt

Hinweise: 
* Für Schatten benötigt man eine Fläche die Schatten abbilden.
* *path tracer* ist wesentlich schöner anzusehen, kostet aber auch sehr viel Rechenleistung.

# Abschluss

1. Speichert ALLE Daten/ Notebooks/ DIAGRAMME
2. Ladet ALLE .ipynb, .npy, .vti herunter
3. Ladet alle .tif Bilder herunter


# Sonstiges

- Wenn Ihr weiter mit Neuronalen Netzwerken/ Python Notebooks experimentieren möchtet, aber keinen Computer mit GPU zur Verfügung habt:
https://colab.research.google.com/notebooks/welcome.ipynb (Account wird benötigt)

- Der Jupyter Notebook Server ist Teil der Anaconda Distribution:
https://www.anaconda.com/distribution/ (ist im Lernzentrum installiert)

- Wenn ihr eine neuere Nvidia Grafikkarte besitzt und diese für das Training von Neuronalen Netzwerken benutzen wollt: 
https://www.nvidia.com/de-de/gpu-cloud/ (Account wird benötigt)

Dieser Notebook Server läuft auf einem Ubuntu System in einem eigenen Docker Container.